# 1. Introduction
This code plots:
1. Monthly Goldstein scale graph (1990-present) covering the one (cluster) of sites in Kosovo
    a.	Two lines on this graph: one representing Goldstein scale overall, one representing Goldstein scale towards IGOs. 
2.	Monthly Goldstein scale graph (1990-present) covering all Libyan World Heritage sites together
    a.	Two lines on this graph: one representing Goldstein scale overall, one representing Goldstein scale towards IGOs. 
3.	Monthly Goldstein scale graph (1990-present) covering all World Heritage sites in Mali together
    a.	Two lines on this graph: one representing Goldstein scale overall, one representing Goldstein scale towards IGOs. 


In [191]:
import pandas as pd
import numpy as np
from dask import dataframe as dd
import xlrd
import math

### 2. Find the PRIO GRID ids for the required states

In [92]:
# Read PRIO GRID data
prio_grid = pd.read_excel("./input/PRIO GRID spine.xls")
prio_grid.index = pd.MultiIndex.from_frame(prio_grid[["lon","lat"]])
tmp = prio_grid.drop(columns = ["lon", "lat"])
tmp.head()

,,gid
lon,lat,
-69.25,-55.75,49182
-68.75,-55.75,49183
-68.25,-55.75,49184
-67.75,-55.75,49185
-67.25,-55.75,49186


In [93]:
# transfer PRIO GRID dataframe to dictionary, the key is long/lat tuple
# the value is PROI GRID ID
prio_dict = tmp.to_dict("dict")
prio_dict

{'gid': {(-69.25, -55.75): 49182,
  (-68.75, -55.75): 49183,
  (-68.25, -55.75): 49184,
  (-67.75, -55.75): 49185,
  (-67.25, -55.75): 49186,
  (-71.25, -55.25): 49898,
  (-70.75, -55.25): 49899,
  (-70.25, -55.25): 49900,
  (-69.75, -55.25): 49901,
  (-69.25, -55.25): 49902,
  (-68.75, -55.25): 49903,
  (-68.25, -55.25): 49904,
  (-67.75, -55.25): 49905,
  (-67.25, -55.25): 49906,
  (-66.75, -55.25): 49907,
  (-66.25, -55.25): 49908,
  (-72.25, -54.75): 50616,
  (-71.75, -54.75): 50617,
  (-71.25, -54.75): 50618,
  (-70.75, -54.75): 50619,
  (-70.25, -54.75): 50620,
  (-69.75, -54.75): 50621,
  (-69.25, -54.75): 50622,
  (-68.75, -54.75): 50623,
  (-68.25, -54.75): 50624,
  (-67.75, -54.75): 50625,
  (-67.25, -54.75): 50626,
  (-66.75, -54.75): 50627,
  (-66.25, -54.75): 50628,
  (-65.75, -54.75): 50629,
  (-65.25, -54.75): 50630,
  (-64.75, -54.75): 50631,
  (-64.25, -54.75): 50632,
  (-63.75, -54.75): 50633,
  (158.75, -54.75): 51078,
  (-73.25, -54.25): 51334,
  (-72.75, -54.25): 5

In [175]:
# calculate the PRIO GRID id based on project's coordination
def coord(lat, long):
    try: 
        # project's original latitude
        x = lat
        # project's original longitude
        y = long
        
        # transfer the original coordination to PRIO GRID's standard coordination
        if x >= 0:
            if x % 1 >= 0 and x % 1 <= 0.5:
                x_2 = (int(x) + 0.25)
            else:
                x_2 = (int(x) + 0.75)
        else:
            if x % 1 >= 0 and x % 1 <= 0.5:
                x_2 = (int(x) - 0.75)
            else:
                x_2 = (int(x) - 0.25)

        if y >= 0:
            if y % 1 >= 0 and y % 1 <= 0.5:
                y_2 = (int(y) + 0.25)
            else:
                y_2 = (int(y) + 0.75)
        else:
            if y % 1 >= 0 and y % 1 <= 0.5:
                y_2 = (int(y) - 0.75)
            else:
                y_2 = (int(y) - 0.25)

        print("original lat",lat)
        print("original long",long)
        print("standard lat",x_2)
        print("standard long",y_2)
        print("------")
        # find the prio grid id based on standard coordination in dictionary (long,lat)
        prio_id = int(prio_dict["gid"][(y_2,x_2)])

    except:
        prio_id = "na"
    
    return prio_id


### 3. Process World Heritage data and find corresponding lat/long with PRIO GRID IDs by region name

In [177]:
# read world heritage data
whc = pd.read_excel("./input/whc-sites-2021 (003).xls")
print(whc.columns)
whc.head(5)

Index(['unique_number', 'id_no', 'rev_bis', 'name_en', 'name_fr',
       'short_description_en', 'short_description_fr', 'justification_en',
       'justification_fr', 'date_inscribed', 'secondary_dates', 'danger',
       'date_end', 'danger_list', 'longitude', 'latitude', 'area_hectares',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'N7', 'N8', 'N9', 'N10',
       'criteria_txt', 'category', 'category_short', 'states_name_en',
       'states_name_fr', 'region_en', 'region_fr', 'iso_code', 'udnp_code',
       'transboundary'],
      dtype='object')


,unique_number,id_no,rev_bis,name_en,name_fr,short_description_en,short_description_fr,justification_en,justification_fr,date_inscribed,...,criteria_txt,category,category_short,states_name_en,states_name_fr,region_en,region_fr,iso_code,udnp_code,transboundary
0,230,208,Rev,Cultural Landscape and Archaeological Remains ...,Paysage culturel et vestiges archéologiques de...,<p>The cultural landscape and archaeological r...,<p>Le paysage culturel et les vestiges archéol...,<p><em>Criterion (i):</em> The Buddha statues ...,<p><em>Critère (i):</em> Les statues de Bouddh...,2003,...,(i)(ii)(iii)(iv)(vi),Cultural,C,Afghanistan,Afghanistan,Asia and the Pacific,Asie et Pacifique,af,afg,0
1,234,211,Rev,Minaret and Archaeological Remains of Jam,Minaret et vestiges archéologiques de Djam,"<p>The 65m-tall Minaret of Jam is a graceful, ...","<p>Haut de 65m, le minaret de Djam est une con...",<p><em>Criterion (ii):</em> The innovative arc...,<p><em>Critère (ii)</em> : L'architecture et l...,2002,...,(ii)(iii)(iv),Cultural,C,Afghanistan,Afghanistan,Asia and the Pacific,Asie et Pacifique,af,afg,0
2,1590,569,Bis,Historic Centres of Berat and Gjirokastra,Centres historiques de Berat et de Gjirokastra,<p>Berat and Gjirokastra are inscribed as rare...,<p>Berat et Gjirokastra sont inscrites en tant...,NaN,NaN,2005,...,(iii)(iv),Cultural,C,Albania,Albanie,Europe and North America,Europe et Amérique du Nord,al,alb,0
3,1563,570,ter,Butrint,Butrint,"<p>Inhabited since prehistoric times, Butrint ...","<p>Habité depuis les temps préhistoriques, le ...",NaN,NaN,1992,...,(iii),Cultural,C,Albania,Albanie,Europe and North America,Europe et Amérique du Nord,al,alb,0
4,111,102,NaN,Al Qal'a of Beni Hammad,La Kalâa des Béni Hammad,<p>In a mountainous site of extraordinary beau...,<p>Dans un site montagneux d&rsquo;une saisiss...,NaN,NaN,1980,...,(iii),Cultural,C,Algeria,Algérie,Arab States,États arabes,dz,dza,0


In [178]:
# find corresponding PRIO GRID ids for each whc based their coordination
whc["prio grid id"] = whc[["latitude", "longitude"]].apply(lambda x: coord(*x), axis=1)
whc.head()

original lat 34.84694
original long 67.82525
standard lat 34.75
standard long 67.75
------
original lat 36.3965555556
original long 64.5160555556
standard lat 36.25
standard long 64.75
------
original lat 40.0741666667
original long 20.1408333333
standard lat 40.25
standard long 20.25
------
original lat 39.745732
original long 20.02095
standard lat 39.75
standard long 20.25
------
original lat 35.81844
original long 4.78684
standard lat 35.75
standard long 4.75
------
original lat 25.5
original long 9.0
standard lat 25.25
standard long 9.25
------
original lat 32.48333
original long 3.68333
standard lat 32.25
standard long 3.75
------
original lat 36.32056
original long 5.73667
standard lat 36.25
standard long 5.75
------
original lat 36.55
original long 2.383333333
standard lat 36.75
standard long 2.25
------
original lat 35.4841666667
original long 6.4688611111
standard lat 35.25
standard long 6.25
------
original lat 36.78333
original long 3.06028
standard lat 36.75
standard long 3

,unique_number,id_no,rev_bis,name_en,name_fr,short_description_en,short_description_fr,justification_en,justification_fr,date_inscribed,...,category,category_short,states_name_en,states_name_fr,region_en,region_fr,iso_code,udnp_code,transboundary,prio grid id
0,230,208,Rev,Cultural Landscape and Archaeological Remains ...,Paysage culturel et vestiges archéologiques de...,<p>The cultural landscape and archaeological r...,<p>Le paysage culturel et les vestiges archéol...,<p><em>Criterion (i):</em> The Buddha statues ...,<p><em>Critère (i):</em> Les statues de Bouddh...,2003,...,Cultural,C,Afghanistan,Afghanistan,Asia and the Pacific,Asie et Pacifique,af,afg,0,179776
1,234,211,Rev,Minaret and Archaeological Remains of Jam,Minaret et vestiges archéologiques de Djam,"<p>The 65m-tall Minaret of Jam is a graceful, ...","<p>Haut de 65m, le minaret de Djam est une con...",<p><em>Criterion (ii):</em> The innovative arc...,<p><em>Critère (ii)</em> : L'architecture et l...,2002,...,Cultural,C,Afghanistan,Afghanistan,Asia and the Pacific,Asie et Pacifique,af,afg,0,181930
2,1590,569,Bis,Historic Centres of Berat and Gjirokastra,Centres historiques de Berat et de Gjirokastra,<p>Berat and Gjirokastra are inscribed as rare...,<p>Berat et Gjirokastra sont inscrites en tant...,NaN,NaN,2005,...,Cultural,C,Albania,Albanie,Europe and North America,Europe et Amérique du Nord,al,alb,0,187601
3,1563,570,ter,Butrint,Butrint,"<p>Inhabited since prehistoric times, Butrint ...","<p>Habité depuis les temps préhistoriques, le ...",NaN,NaN,1992,...,Cultural,C,Albania,Albanie,Europe and North America,Europe et Amérique du Nord,al,alb,0,186881
4,111,102,NaN,Al Qal'a of Beni Hammad,La Kalâa des Béni Hammad,<p>In a mountainous site of extraordinary beau...,<p>Dans un site montagneux d&rsquo;une saisiss...,NaN,NaN,1980,...,Cultural,C,Algeria,Algérie,Arab States,États arabes,dz,dza,0,181090


In [179]:
# get the PRIO GRID ids for WHC in Mali
whc_Mali = whc[whc["states_name_en"]=="Mali"][["states_name_en","prio grid id"]]
print(whc_Mali.shape)
whc_Mali_prio_id = whc_Mali["prio grid id"].tolist()
whc_Mali_prio_id

(4, 2)


[149391, 153715, 150114, 153000]

In [180]:
# get the PRIO GRID id for WHC in Libyan
whc_Libyan = whc[whc["states_name_en"]=="Libya"][["states_name_en","prio grid id"]]
print(whc_Libyan.shape)
whc_Libyan_prio_id = whc_Libyan["prio grid id"].tolist()
whc_Libyan_prio_id

(5, 2)


[176789, 176785, 176804, 165261, 173179]

In [181]:
# get the PRIO GRID ids for sites in Kosovo by this link: https://whc.unesco.org/en/list/724/maps/
coord_kosovo = [(42.5335, 20.2668), (42.6501, 20.2500), (42.2001, 20.7332),(42.5835, 21.1833)]
whc_Kosovo_prio_id = []
for i in coord_kosovo:
    whc_Kosovo_prio_id.append(coord(i[0], i[1]))
whc_Kosovo_prio_id

original lat 42.5335
original long 20.2668
standard lat 42.75
standard long 20.25
------
original lat 42.6501
original long 20.25
standard lat 42.75
standard long 20.25
------
original lat 42.2001
original long 20.7332
standard lat 42.25
standard long 20.75
------
original lat 42.5835
original long 21.1833
standard lat 42.75
standard long 21.25
------


[191201, 191201, 190482, 191203]

### 4. Find the corresponding Goldstein Scale in corresponding areas

In [135]:
# get a glimpse of the monthly data by Dask
monthly_Data = dd.read_csv("d:/goldstein_index_1990_to_2022_monthly.csv")
monthly_Data.head()

,gid,month_year,AGR_All_Actors_Count,AGR_All_Actors_GoldsteinScale,BUS_All_Actors_Count,BUS_All_Actors_GoldsteinScale,BUS_MNC_All_Actors_Count,BUS_MNC_All_Actors_GoldsteinScale,COP_All_Actors_Count,COP_All_Actors_GoldsteinScale,...,All_Actors_REF_Count,All_Actors_REF_GoldsteinScale_Count,All_Actors_SEP_Count,All_Actors_SEP_GoldsteinScale_Count,All_Actors_SPY_Count,All_Actors_SPY_GoldsteinScale_Count,All_Actors_UAF_Count,All_Actors_UAF_GoldsteinScale_Count,All_Actors_UIS_Count,All_Actors_UIS_GoldsteinScale_Count
0,49182,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49183,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49184,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49185,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49186,1990-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
# print the variable names 
for i in monthly_Data.columns:
    print(i)

gid
month_year
AGR_All_Actors_Count
AGR_All_Actors_GoldsteinScale
BUS_All_Actors_Count
BUS_All_Actors_GoldsteinScale
BUS_MNC_All_Actors_Count
BUS_MNC_All_Actors_GoldsteinScale
COP_All_Actors_Count
COP_All_Actors_GoldsteinScale
COP_JUD_MIL_SPY_All_Actors_Count
COP_JUD_MIL_SPY_All_Actors_GoldsteinScale
CRM_All_Actors_Count
CRM_All_Actors_GoldsteinScale
CVL_All_Actors_Count
CVL_All_Actors_GoldsteinScale
EDU_All_Actors_Count
EDU_All_Actors_GoldsteinScale
ELI_All_Actors_Count
ELI_All_Actors_GoldsteinScale
ENV_All_Actors_Count
ENV_All_Actors_GoldsteinScale
GOV_All_Actors_Count
GOV_All_Actors_GoldsteinScale
HLH_All_Actors_Count
HLH_All_Actors_GoldsteinScale
HRI_All_Actors_Count
HRI_All_Actors_GoldsteinScale
IGO_All_Actors_Count
IGO_All_Actors_GoldsteinScale
IMG_All_Actors_Count
IMG_All_Actors_GoldsteinScale
INS_All_Actors_Count
INS_All_Actors_GoldsteinScale
INS_REB_IMG_All_Actors_Count
INS_REB_IMG_All_Actors_GoldsteinScale
INT_All_Actors_Count
INT_All_Actors_GoldsteinScale
INT_UIS_All_Actors_

In [137]:
# only read part of the data with useful variables
tables = pd.read_csv("d:/goldstein_index_1990_to_2022_monthly.csv", usecols = ["gid", "month_year", "All_Actors_IGO_GoldsteinScale_Count"])
tables.to_csv("output/small goldstein table with 3 vairbale.csv")

##### Libyan Data

In [156]:
tables_Libyan = tables.loc[tables["gid"].isin(whc_Libyan_prio_id)]
tables_Libyan.to_csv("output/tables_Libyan.csv")

,gid,month_year,All_Actors_IGO_GoldsteinScale_Count
23921,165261,1990-01,NaN
27309,173179,1990-01,NaN
28969,176785,1990-01,NaN
28973,176789,1990-01,NaN
28978,176804,1990-01,NaN
...,...,...,...
25627031,165261,2022-12,NaN
25630419,173179,2022-12,NaN
25632079,176785,2022-12,NaN
25632083,176789,2022-12,NaN


In [158]:
sort_by_month_libyan = tables_Libyan.groupby(by = ["month_year"])["All_Actors_IGO_GoldsteinScale_Count"].sum()
sort_by_month_libyan.to_csv("output/time_series_libyan.csv")

##### Mali Data

In [195]:
tables_mali = tables.loc[tables["gid"].isin(whc_Mali_prio_id)]
tables_mali.to_csv("output/tables_mali.csv")
tables_mali

,gid,month_year,All_Actors_IGO_GoldsteinScale_Count
18503,149391,1990-01,NaN
18705,150114,1990-01,NaN
19534,153000,1990-01,NaN
19745,153715,1990-01,NaN
83321,149391,1990-02,NaN
...,...,...,...
25558037,153715,2022-11,NaN
25621613,149391,2022-12,NaN
25621815,150114,2022-12,NaN
25622644,153000,2022-12,NaN


In [161]:
sort_by_month_mali= tables_mali.groupby(by = ["month_year"])["All_Actors_IGO_GoldsteinScale_Count"].sum()
sort_by_month_mali.to_csv("output/time_series_mali.csv")

#### Kosovo Data

In [196]:
tables_kosovo = tables.loc[tables["gid"].isin(whc_Kosovo_prio_id)]
tables_kosovo.to_csv("output/tables_kosovo.csv")
tables_kosovo

,gid,month_year,All_Actors_IGO_GoldsteinScale_Count
35365,190482,1990-01,NaN
35720,191201,1990-01,NaN
35722,191203,1990-01,NaN
100183,190482,1990-02,NaN
100538,191201,1990-02,NaN
...,...,...,...
25574012,191201,2022-11,NaN
25574014,191203,2022-11,NaN
25638475,190482,2022-12,NaN
25638830,191201,2022-12,NaN


In [163]:
sort_by_month_kosovo= tables_kosovo.groupby(by = ["month_year"])["All_Actors_IGO_GoldsteinScale_Count"].sum()
sort_by_month_kosovo.to_csv("output/time_series_kosovo.csv")